# Naive Bayes
朴素贝叶斯是贝叶斯决策理论的一部分，所以讲述朴素贝叶斯之前有必要了解一下贝叶斯理论决策。<br>
假设有一个数据集，它由两类数据组成：
![](imgs/bys.png)
现在，有一个点$（x,y）$需要我们来判断是属于红色和蓝色中的那一类。我们用 $p红(x,y)$ 表示这个点属于红色分类的概率， $p蓝(x,y)$ 表示这个点属于蓝色分类的概率。那么如果：
* $p红(x,y)>p蓝(x,y)$,这个点就属于红色分类
* $p红(x,y)<p蓝(x,y)$,这个点就属于蓝色分类

接下来，我们需要回顾一下概率基本知识。
## 概率基本知识
朴素贝叶斯算法是一个基于概率论的分类方法：
* 先验概率：<br>
    某个事件发生的概率，与其他事件无关。事件A的先验概率记为$P(A)$<br><br>
* 条件概率：<br>
    事件A在另一个事件B已经发生的条件下发生的概率，记作<br><br> $$P(A|B)=\frac{P(AB)}{P(B)}$$<br>
* 联合概率：<br>
    表示两个事件共同发生的概率。$A$与$B$的联合概率表示为$P(AB)$或者$P(A,B)$或者$P(A\cap B)$<br><br>
* 全概率公式：<br>
    + 假设样本事件组{$B_i$}是空间$\Omega$的一个划分（$B_1\cup B_2 \cup...B_n=\Omega$），对于任一事件$A$在样本空间发生的概率为：
    
    $$\begin{align}
P(A)&=P(AB_1)+P(AB_2)+...+P(AB_n)\\ &=P(B_1)P(A|B_1)+P(B_2)P(A|B_2)+...+P(B_n)P(A|B_n)\\ &=\sum_{i=1}^nP(B_i)P(A|B_i)
\end{align}$$

    + 全概率公式的意义在于，当直接计算$P(A)$较为困难，而$P(B_i)$，$P(A|B_i)$  的计算较为简单时，先找到样本空间$\Omega$的一个个划分$B_1,B_2,...B_n$，这样事件A的概率就为与样本空间的所有划分同时发生的概率和

> <span style="color:red;">练习：</span>某车间用甲、乙、丙三台机床进行生产，各台机床次品率分别为3%，3%，1%，它们各自的产品分别占总量的30%，20%，50%，将它们的产品混在一起，求任取出一个产品为次品的概率？<br><br>
$\begin{align}P(A) &= P(B_甲)P(A|B_甲)+P(B_乙)P(A|B_乙)+P(B_丙)P(A|B_丙)\\
 &= 0.3\times0.03+0.2\times0.03+0.5\times0.01 \\&= 0.02\end{align}$


## 贝叶斯公式
贝叶斯公式是建立在条件概率的基础上寻找事件发生的原因。设$B_1,B_2,...B_n$是样本空间Ω的一个划分，则对任一事件$A（P(A)>0)$，有：<br><br>
$$P(B_i|A)=\frac{P(B_i)P(A|B_i)}{P(A)}=\frac{P(B_i)P(A|B_i)}{\sum \limits_{j=1}^nP(B_j)P(A|B_j)}$$

> <span style="color:red;">练习：</span>某车间用甲、乙、丙三台机床进行生产，各台机床次品率分别为3%，3%，1%，它们各自的产品分别占总量的30%，20%，50%，现在取出一个次品，求该次品最有可能来自甲、乙、丙中哪一台？<br><br>
$P(B_甲|A) = \frac{P(B_甲)P(A|B_甲)}{P(A)} = \frac{0.3\times0.03}{0.02} = 0.45$<br>
$P(B_乙|A) = \frac{P(B_乙)P(A|B_乙)}{P(A)} = \frac{0.2\times0.03}{0.02}=0.3$<br>
$P(B_丙|A) = \frac{P(B_丙)P(A|B_丙)}{P(A)} = \frac{0.5\times0.01}{0.02}=0.25$<br>

## 朴素贝叶斯
之所以称为“朴素”，是因为做了个假设：样本的特征之间相互独立。假设事件$A$由 $\{A_1,A_2,…,A_n\}$ 组成，$\{A_1,A_2,…,A_n\}$就对应一个样本的$n$个特征，由于各特征相互独立，所以$$P(A|B_j) = P(A_1|B_j)P(A_2|B_j)...P(A_n|B_j) = \prod_{i=1}^nP(A_i|B_j)$$
$$P(B_i|A)=\frac{P(B_i)P(A|B_i)}{P(A)}=\frac{P(B_i)\prod \limits_{i=1}^nP(A_i|B_j)}{P(A)}$$
也许这样理解更好点：

$$
P(类别|特征)=\frac{P(特征|类别)*P(类别)}{P(特征)}
$$




## 朴素贝叶斯示例1：过滤网站恶意留言


In [1]:
from numpy import *
#加载样本数据
def loadDataSet():
    postingList = [['my', 'dog', 'has', 'flea', 'problems', 'help', 'please'],
                   ['maybe', 'not', 'take', 'him', 'to', 'dog', 'park', 'stupid'],
                   ['my', 'dalmation', 'is', 'so', 'cute', 'I', 'love', 'him'],
                   ['stop', 'posting', 'stupid', 'worthless', 'garbage'],
                   ['mr', 'licks', 'ate', 'my', 'steak', 'how', 'to', 'stop', 'him'],
                   ['quit', 'buying', 'worthless', 'dog', 'food', 'stupid']]
    classVec = [0, 1, 0, 1, 0, 1] # 0 表示正常言论，1表示恶意言论
    return postingList, classVec 
 
#合并所有单词，利用set来去重，得到所有单词的唯一列表
def createVocabList(dataSet):
    #创建一个空集
    vocabSet = set([])
    for document in dataSet:
        #|得到2个集合的并集
        vocabSet = vocabSet | set(document)
    return list(vocabSet) #得到去重后的所有单词

#构建词向量
#词袋模型，将单词列表变为数字向量列表，对应数字表示出现的次数
def bagOfWords2VecMN(vocabList, inputSet):
    returnVec = [0] * len(vocabList)  #获得与所有单词等长的元素为0的列表   
    for word in inputSet:             #遍历要判断的单词集合
        if word in vocabList:        #出现和样本数据里对应单词，出现次数加1    
            returnVec[vocabList.index(word)] += 1    
    return returnVec
 
# 返回的是0、1各自两个分类中每个单词数量除以该分类单词总量再取对数ln 以及1类的比例
def trainNB0(trainMatrix, trainCategory):#trainMatrix文档矩阵，trainCategory标签向量
    
    
    numTrainDocs = len(trainMatrix)  # 样本数
    numWords = len(trainMatrix[0])  # 特征数
    pAbusive = sum(trainCategory) / float(numTrainDocs)  # 1类所占比例=所有的1加起来/总样本数 
    p0Num = ones(numWords)
    p1Num = ones(numWords)  #初始化所有单词为1
    p0Denom = 2.0
    p1Denom = 2.0  #初始化总单词为2     
    
    
    for i in range(numTrainDocs):
        if trainCategory[i] == 1:       #求1类
            p1Num += trainMatrix[i]      #对应单词次数增加
            p1Denom += sum(trainMatrix[i]) #总单词次数增加
        else:                           #求0类
            p0Num += trainMatrix[i]     
            p0Denom += sum(trainMatrix[i])
    p1Vect = log(p1Num / p1Denom)  # numpy数组 / float = 1中每个单词/1中总单词
    p0Vect = log(p0Num / p0Denom)  # 这里为什么还用ln来处理，后面说明
    return p0Vect, p1Vect, pAbusive
 
#P(X|C)判断各类别的概率大小（这里是0、1）
def classifyNB(vec2Classify, p0Vec, p1Vec, pClass1):
    p1 = sum(vec2Classify * p1Vec) + log(pClass1)  # 相乘后得到哪些单词存在，再求和，log(P(Ai|Bi))+log(P(Bi))
    p0 = sum(vec2Classify * p0Vec) + log(1.0 - pClass1) # 由于使用的是ln，这里其实都是对数相加
    if p1 > p0:
        return 1
    else:
        return 0
    
#封装测试函数
def testingNB():
    listOPosts, listClasses = loadDataSet()
    myVocabList = createVocabList(listOPosts)
    trainMat = []
    for postinDoc in listOPosts:
        trainMat.append(bagOfWords2VecMN(myVocabList, postinDoc))
    p0V, p1V, pAb = trainNB0(array(trainMat), array(listClasses))
    #上面求出了0、1两个类中各单词所占该类的比例，以及0、1的比例
 
    #下面是预测两条样本数据的类别
    testEntry1 = ['love', 'my', 'dalmation']
    thisDoc1 = array(bagOfWords2VecMN(myVocabList, testEntry1)) #先将测试数据转为numpy的词袋模型 [0 2 0 5 1 0 0 3 ...]
    print(testEntry1, 'classified as: ', classifyNB(thisDoc1, p0V, p1V, pAb)) #传值判断
 
    testEntry2 = ['stupid', 'garbage']
    thisDoc2 = array(bagOfWords2VecMN(myVocabList, testEntry2))
    print(testEntry2, 'classified as: ', classifyNB(thisDoc2, p0V, p1V, pAb))
    
#调用测试
testingNB()

['love', 'my', 'dalmation'] classified as:  0
['stupid', 'garbage'] classified as:  1


In [2]:
a,b=loadDataSet()
createVocabList(a)

['food',
 'how',
 'take',
 'has',
 'him',
 'please',
 'buying',
 'so',
 'dalmation',
 'love',
 'cute',
 'licks',
 'posting',
 'mr',
 'stop',
 'help',
 'I',
 'flea',
 'my',
 'problems',
 'to',
 'quit',
 'maybe',
 'garbage',
 'stupid',
 'steak',
 'ate',
 'park',
 'dog',
 'is',
 'worthless',
 'not']

## 朴素贝叶斯的应用示例2——英文单词拼写检查
#### 原理
用户输入了一个单词。这时分成两种情况：拼写正确或者拼写不正确。我们把拼写正确的情况记做c（代表correct），拼写错误的情况记做w（代表wrong）。

所谓"拼写检查"，就是在发生 w 的情况下，试图推断出 c 。从概率论的角度看，就是已知 w ，然后在若干个备选方案中，找出可能性最大的那个 c ，也就是求 P(c|w) 的最大值。

根据贝叶斯定理：P(c|w) = P(w|c) * P(c) / P(w)，对于所有备选的 c 来说，对应的都是同一个 w ，所以它们的 P(w) 是相同的，因此我们求的其实是分子<span style="color:red"> P(w|c) * P(c) </span>最大值。

<span style="color:red">P(c)</span>：某个正确的词的出现"概率"，它可以用"频率"代替。如果我们有一个足够大的文本库，那么这个文本库中每个单词的出现频率，就相当于它的发生概率。某个词的出现频率越高，P(c) 就越大。

<span style="color:red">P(w|c)</span>：在试图拼写 c 的情况下，出现拼写错误 w 的概率。这个概率需要统计数据的支持，但是为了简化问题，我们可以假设两个单词在字形上越接近，就有越可能拼错，P(w|c) 就越大。具体来说，相差一个字母的拼法，就比相差两个字母的拼法，发生概率更高。你想拼写单词hello，那么错误拼成 helli（相差一个字母）的可能性，就比拼成 hellii高（相差两个字母）。

所以，我们只要找到<span style="color:red">与输入单词在字形上最相近的那些词，再在其中挑出出现频率最高的一个</span>，就能实现 P(w|c) * P(c) 的最大值。

#### 算法实现
最简单的算法，只需要4步：
* 建立单词库
* 统计单词库中每个单词出现的频率
* 根据用户输入的单词，得到所有可能的相似拼写
    + "相似拼写"：指的是两个单词之间的"编辑距离"（edit distance）不超过2。也就是说，两个词只相差1到2个字母，通过删除、交换、更改和插入这四种操作中的一种，就可以让一个词变成另一个词。
    
* 比较所有拼写相近的词在文本库的出现频率。频率最高的那个词，就是正确的拼法。


In [1]:
from collections import Counter
?Counter
# import re
# ?re.findall

In [3]:
import re
from collections import Counter

def words(text): return re.findall(r'\w+', text.lower())

#Counter类返回一个以元素为 key 、元素个数为 value 的 Counter 对象集合
WORDS = Counter(words(open("./data/words.txt").read()))

#注意这里必须是python原生的sum 不能是numpy的sum
#如果先前引入了numpy 请重启Kernel
def P(word, N=sum(WORDS.values())):
    return WORDS[word] / N

def known(words):
    return set(w for w in words if w in WORDS)

def candidates(word):
    return (known([word]) or known(edits1(word)) or known(edits2(word)) or [word])

def edits1(word):
    letters = 'abcdefghijklmnopqrstuvwxyz'

    # 将word依次按照每一位分割成前后两半。比如，'abc'会被分割成 [('', 'abc'), ('a', 'bc'), ('ab', 'c'), ('abc', '')]
    splits = [(word[:i], word[i:]) for i in range(len(word) + 1)]

    # 依次删除word的每一位后、所形成的所有新词。比如，'abc'对应的就是 ['bc', 'ac', 'ab']
    d = [L + R[1:] for L, R in splits if R]

    # 依次交换word的邻近两位，所形成的所有新词。比如，'abc' 对应的就是['bac', 'acb']
    t = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R) > 1]

    # 将word的每一位依次替换成其他25个字母，所形成的所有新词。比如，'abc'对应的就是 ['abc', 'bbc', 'cbc', ... , 'abx', ' aby', 'abz' ]
    r = [L + c + R[1:] for L, R in splits if R for c in letters]

    # 在word的邻近两位之间依次插入一个字母，所形成的所有新词。比如，'abc' 对应的就是['aabc', 'babc', 'cabc', ..., 'abcx', 'abcy', 'abcz']
    i = [L + c + R for L, R in splits for c in letters]
    return set(d + t + r + i)

def edits2(word):
    return (e2 for e1 in edits1(word) for e2 in edits1(e1))

#测试方式
def correction(word):
    # return max(candidates(word), key=P)
    print(f"你的原始输入是： {word} 你可能想输入的是： {max(candidates(word), key=P)}")

correction("tttttt")

你的原始输入是： tttttt 你可能想输入的是： tttttt


In [11]:
WORDS = Counter(words(open("./data/test.txt").read()))
print(WORDS)
print(type(sum(WORDS.values())))

Counter({'the': 6, 'of': 5, 'in': 3, 'to': 3, 'something': 3, 'has': 2, 'a': 2, 'may': 2, 'be': 2, 'life': 2, 'oneself': 2, 'my': 2, 'mind': 2, 'feel': 2, 'do': 2, 'ge': 1, 'reached': 1, 'end': 1, 'beginning': 1, 'word': 1, 'guilty': 1, 'his': 1, 'seems': 1, 'passing': 1, 'lot': 1, 'different': 1, 'became': 1, 'appearance': 1, 'same': 1, 'day': 1, 'back': 1, 'past': 1, 'paranoid': 1, 'weird': 1, 'belief': 1, 'disillusionment': 1, 'these': 1, 'days': 1, 'been': 1, 'very': 1, 'messy': 1, 'constantly': 1, 'always': 1, 'should': 1, 'go': 1, 'or': 1, 'write': 1, 'twenty': 1, 'years': 1, 'trajectory': 1, 'deeply': 1, 'shallow': 1, 'suddenly': 1, 'it': 1})
<class 'dict_values'>
